# Dataset Prompt Engineering & Description Creation

## Setup

In [1]:
!pip install openai
!pip install retry

In [17]:
import os
import time
from tqdm.notebook import tqdm
from retry import retry

import pandas as pd

import openai

In [18]:
openai_key = os.environ.get("OPENAI_KEY")

openai.api_key = openai_key

## Dataset Processing

In [19]:
input_path = './data/master_midi_meta_ken.csv' # UPDATE
output_path = './data/master_midi_output_ken.csv' # UPDATE

df = pd.read_csv(input_path)

#### Dataset Sanitization

In [20]:
def add_space(text):
    note = text[0]
    key = text[-5:]
    return note + ' ' + key

def genre_split(text):
    if text == 'newage':
        text = 'new age'
    if text == 'posthardcore':
        text = 'post hardcore'
    if text == 'easylistening':
        text = 'easy listening'
    return text

In [21]:
# Data grooming
df['audio_key'] = df['audio_key'].apply(add_space)
df['pitch_range'] = df['pitch_range'].str.replace('_', ' ')
df['genre'] = df['genre'].apply(genre_split).str.replace('__', ' ').replace('_',' ')
df['inst'] = df['inst'].str.replace('_', ' ').replace('-', ' ')
df['track_role'] = df['track_role'].str.replace('_', ' ')

In [22]:
df.head()

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note
0,e minor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G',...",unknown,6,231,rock,unknown,clarinet,unknown,4/4,66,100,unknown,c527f2542d8533c1a4c95b2217d9918f_15,unknown,"['E', 'Em', 'G', 'A']"
1,a minor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'A',...",unknown,9,222,rock,unknown,recorder,unknown,4/4,66,100,unknown,c527f2542d8533c1a4c95b2217d9918f_16,unknown,"['B', 'E', 'C', 'A']"
2,d minor,"[['A', 'A', 'A', 'A', 'G', 'G', 'G', 'G', 'A',...",unknown,9,235,rock,unknown,electric bass pick,unknown,4/4,100,127,unknown,c527f2542d8533c1a4c95b2217d9918f_2,unknown,"['E', 'C', 'G', 'A']"
3,a minor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",unknown,10,213,rock,unknown,acoustic guitar nylon,unknown,4/4,23,74,unknown,c527f2542d8533c1a4c95b2217d9918f_3,unknown,"['D', 'G', 'F', 'E', 'Am']"
4,a minor,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',...",unknown,10,211,rock,unknown,electric guitar muted,unknown,4/4,55,127,unknown,c527f2542d8533c1a4c95b2217d9918f_4,unknown,"['B', 'D', 'G', 'F', 'Em']"


In [23]:
# Split main dataset by number of rows (rate limit)

num_rows = 2297 # Update to max rate limit
len_df = len(df)

df_list = []

for i in range(0, len_df, num_rows):
    df_append = df[i:i + num_rows]
    df_list.append(df_append)

len(df_list)

7

## Prompt Engineering

- Prompt engineering performed directly in ChatGPT


##### **4K context**
    - Input: $0.0015 per 1K tokens
    - Output: $0.002 per 1K tokens

    ~$0.000325 per query * ~15K rows = $4.88

#### **Prompt:**

**PROMPT 1:** 

Pretend you are a musician that wants to have fun jamming to music produced by a set of instructions. Write these instructions in 25 to 50 words, where the output music incorporates the following features: a minor key, mid pitch range, 8 measures, 120 beats/minute, cinematic genre, unknown track role,  bass instrument, standard sample rhythm, 4/4 time signature, minimum velocity of 101, maximum velocity of 102, and unique chords of ['D', 'Dm', 'G', 'C', 'Am']. The unique chords feature represents a list of unique chords. If a feature includes the word 'unknown', then eliminate this feature from your response.

**PROMPT 2:**

Give a realistic but natural instruction between 25 to 50 words in the point-of-view of a hobbyist musician doing a jamming or accompaniment session, which includes the following features: a minor key, mid pitch range, 8 measures, 120 beats/minute, cinematic genre, unknown track role,  bass instrument, standard sample rhythm, 4/4 time signature, minimum velocity of 101, maximum velocity of 102, and unique chords of ['D', 'Dm', 'G', 'C', 'Am']. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.

**PROMPT 3:**

You are a professional musician looking to sharpen musical skills through an accompaniment session. Write a set of instructions between 25 to 50 words long that will generate music based on the following features: a minor key, mid pitch range, 8 measures, 120 beats/minute, cinematic genre, unknown track role,  bass instrument, standard sample rhythm, 4/4 time signature, minimum velocity of 101, maximum velocity of 102, and unique chords of ['D', 'Dm', 'G', 'C', 'Am']. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.

**PROMPT 4:**

Give a user, who wants to try AI for generating music, an example prompt between 25 to 50 words long that will incorporate the following features into a piece of music: a minor key, mid pitch range, 8 measures, 120 beats/minute, cinematic genre, unknown track role,  bass instrument, standard sample rhythm, 4/4 time signature, minimum velocity of 101, maximum velocity of 102, and unique chords of ['D', 'Dm', 'G', 'C', 'Am']. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.

**PROMPT 5:**

Can you organize the following features into a set of instructions between 25 to 50 words long that can be used to prompt an LLM to produce a piece of music: a minor key, mid pitch range, 8 measures, 120 beats/minute, cinematic genre, unknown track role,  bass instrument, standard sample rhythm, 4/4 time signature, minimum velocity of 101, maximum velocity of 102, and unique chords of ['D', 'Dm', 'G', 'C', 'Am']. The unique chords feature represents a list of unique chords. Your response should include music terms, and be experimental and thematic. If a feature includes the word 'unknown', then eliminate this feature from your response.

## ChatGPT Response Generation

In [24]:
@retry(Exception, tries=5, delay=1, backoff=2, max_delay=120)
def get_completion(prompt, model='gpt-3.5-turbo'):
    messages = [{'role': 'user', 'content': prompt}]

    response = openai.ChatCompletion.create(model=model, 
                                            messages=messages,
                                           )
    
    return response.choices[0].message['content']

In [25]:
# Manually update the dataframe
df = df_list[1]

print(f"Dataframe length: {len(df)}")

Dataframe length: 2297


In [26]:
temperature = 0.3
sleep_delay = 0.12

text_list = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    
    # Update prompts based on prompt engineering
    if idx % 10 == 4 or idx % 10 == 9:
        # prompt 1
        prompt = "Pretend you are a musician that wants to have fun jamming to music produced by a set of instructions. Write these instructions in 25 to 50 words, where the output music incorporates the following features: {} key, {} pitch range, {} measures, {} beats/minute, {} genre, {} track role,  {} instrument, {} sample rhythm, {} time signature, minimum velocity of {}, maximum velocity of {}, and unique chords of {}. The unique chords feature represents a list of unique chords. If a feature includes the word 'unknown', then eliminate this feature from your response.".format(
                        row['audio_key'], 
                        row['pitch_range'], 
                        row['num_measures'], 
                        row['bpm'], 
                        row['genre'], 
                        row['track_role'], 
                        row['inst'], 
                        row['sample_rhythm'],
                        row['time_signature'], 
                        row['min_velocity'], 
                        row['max_velocity'], 
                        row['unique_chord_n_note']
                        )
    
    if idx % 10 == 0 or idx % 10 == 7:
        # prompt 2
        prompt = "Give a realistic but natural instruction between 25 to 50 words in the point-of-view of a hobbyist musician doing a jamming or accompaniment session, which includes the following features: {} key, {} pitch range, {} measures, {} beats/minute, {} genre, {} track role,  {} instrument, {} sample rhythm, {} time signature, minimum velocity of {}, maximum velocity of {}, and unique chords of {}. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.".format(
                        row['audio_key'], 
                        row['pitch_range'], 
                        row['num_measures'], 
                        row['bpm'], 
                        row['genre'], 
                        row['track_role'], 
                        row['inst'], 
                        row['sample_rhythm'],
                        row['time_signature'], 
                        row['min_velocity'], 
                        row['max_velocity'], 
                        row['unique_chord_n_note']
                        )
        
    if idx % 10 == 2 or idx % 10 == 8:
        # prompt 3
        prompt = "You are a professional musician looking to sharpen musical skills through an accompaniment session. Write a set of instructions between 25 to 50 words long that will generate music based on the following features: {} key, {} pitch range, {} measures, {} beats/minute, {} genre, {} track role,  {} instrument, {} sample rhythm, {} time signature, minimum velocity of {}, maximum velocity of {}, and unique chords of {}. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.".format(
                        row['audio_key'], 
                        row['pitch_range'], 
                        row['num_measures'], 
                        row['bpm'], 
                        row['genre'], 
                        row['track_role'], 
                        row['inst'], 
                        row['sample_rhythm'],
                        row['time_signature'], 
                        row['min_velocity'], 
                        row['max_velocity'], 
                        row['unique_chord_n_note']
                        )        

    if idx % 10 == 1 or idx % 10 == 6:
        # prompt 4
        prompt = "Give a user, who wants to try AI for generating music, an example prompt between 25 to 50 words long that will incorporate the following features into a piece of music: {} key, {} pitch range, {} measures, {} beats/minute, {} genre, {} track role,  {} instrument, {} sample rhythm, {} time signature, minimum velocity of {}, maximum velocity of {}, and unique chords of {}. The unique chords feature represents a list of unique chords. Your response should include music terms. If a feature includes the word 'unknown', then eliminate this feature from your response.".format(
                        row['audio_key'], 
                        row['pitch_range'], 
                        row['num_measures'], 
                        row['bpm'], 
                        row['genre'], 
                        row['track_role'], 
                        row['inst'], 
                        row['sample_rhythm'],
                        row['time_signature'], 
                        row['min_velocity'], 
                        row['max_velocity'], 
                        row['unique_chord_n_note']
                        )  
        
    if idx % 10 == 3 or idx % 10 == 5:
        # prompt 5
        prompt = "Can you organize the following features into a set of instructions between 25 to 50 words long that can be used to prompt an LLM to produce a piece of music: {} key, {} pitch range, {} measures, {} beats/minute, {} genre, {} track role,  {} instrument, {} sample rhythm, {} time signature, minimum velocity of {}, maximum velocity of {}, and unique chords of {}. The unique chords feature represents a list of unique chords. Your response should include music terms, and be experimental and thematic. If a feature includes the word 'unknown', then eliminate this feature from your response.".format(
                        row['audio_key'], 
                        row['pitch_range'], 
                        row['num_measures'], 
                        row['bpm'], 
                        row['genre'], 
                        row['track_role'], 
                        row['inst'], 
                        row['sample_rhythm'],
                        row['time_signature'], 
                        row['min_velocity'], 
                        row['max_velocity'], 
                        row['unique_chord_n_note']
                        )  
    
    response = get_completion(prompt)
    text_list.append(response)
    time.sleep(sleep_delay)

  0%|          | 0/2297 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
num_text = len(text_list)
print(f"Number of records: {num_text}")
df_limit = df.head(num_text)

Number of records: 1681


In [13]:
df_limit.loc[:, 'text'] = text_list

C:\Users\ktrin\AppData\Local\Temp\ipykernel_16476\2290908449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limit.loc[:, 'text'] = text_list


In [14]:
df_limit.count()

audio_key              1681
chord_progressions     1681
pitch_range            1681
num_measures           1681
bpm                    1681
genre                  1681
track_role             1681
inst                   1681
sample_rhythm          1681
time_signature         1681
min_velocity           1681
max_velocity           1681
split_data             1681
id                     1681
track_roll             1681
unique_chord_n_note    1681
text                   1681
dtype: int64

In [15]:
df_limit

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,text
617,f major,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A',...",unknown,30,76,soul,unknown,trumpet,unknown,4/4,69,112,unknown,c7c248177811a364ce7b5a90c5e75903_5,unknown,"['B', 'C', 'F', 'A']",Let's start a soul jam session in F major. We'...
618,d minor,"[['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'C',...",unknown,32,69,soul,unknown,orchestra hit,unknown,4/4,81,115,unknown,c7c248177811a364ce7b5a90c5e75903_6,unknown,"['D', 'Dm', 'G', 'F', 'C', 'E', 'A']",Set your tempo to 69 beats per minute and time...
619,f major,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A',...",unknown,14,139,soul,unknown,acoustic bass,unknown,4/4,100,112,unknown,c7c248177811a364ce7b5a90c5e75903_8,unknown,"['B', 'D', 'G', 'C', 'F', 'A']",I will gladly jam along with these instruction...
620,c major,"[['F', 'F', 'F', 'F', 'C', 'C', 'C', 'C', 'D',...",unknown,8,167,soul,unknown,alto sax,unknown,4/4,89,107,unknown,c7c248177811a364ce7b5a90c5e75903_9,unknown,"['D', 'G', 'C', 'F', 'E']",Let's start a jamming session in C major. Set ...
621,b minor,"[['B', 'B', 'B', 'B', 'E', 'E', 'E', 'E', 'E',...",unknown,19,112,rock,unknown,electric bass finger,unknown,4/4,127,127,unknown,c7c7692a06514baf215da8d382a95faa_1,unknown,"['B', 'F#', 'Bm', 'C#', 'E']",Compose a 35-measure rock piece in the key of ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,a minor,"[['A', 'A', 'A', 'A', 'C#', 'C#', 'C#', 'C#', ...",unknown,13,110,rock,unknown,acoustic bass,unknown,4/4,84,114,unknown,cf5be7bbe3e085a9c170fd684861f3c6_9,unknown,"['B', 'D', 'G', 'C', 'C#', 'E', 'A']",Compose a 13-measure experimental rock piece i...
2294,d minor,"[['F', 'F', 'F', 'F', 'Fm', 'Fm', 'Fm', 'Fm', ...",unknown,2,240,pop,unknown,choir aahs,unknown,4/4,73,92,unknown,cf611cfa8105e5bb289f7003da018499_10,unknown,"['Cm', 'C', 'F', 'Fm']",Have a blast jamming to pop music in d minor k...
2295,d minor,"[['E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'Dm'...",unknown,5,240,pop,unknown,celesta,unknown,4/4,127,127,unknown,cf611cfa8105e5bb289f7003da018499_11,unknown,"['E', 'G', 'F', 'Dm']",Create a pop genre piece in d minor key with a...
2296,f major,"[['F', 'F', 'F', 'F', 'Am', 'Am', 'Am', 'Am', ...",unknown,17,60,pop,unknown,pad 1 new age,unknown,4/4,72,92,unknown,cf611cfa8105e5bb289f7003da018499_13,unknown,"['B', 'Cm', 'F', 'Am']",Create a dynamic and catchy pop composition in...


## Dataset Update

In [16]:
if os.path.exists(output_path):
    csv_df = pd.read_csv(output_path)
    if csv_df.empty:
        df_limit.to_csv(output_path, index=False)
    else:
        concat_df = pd.concat([csv_df, df_limit])
        concat_df.to_csv(output_path, index=False)
else:
    df_limit.to_csv(output_path, index=False)
